In [ ]:
from googleapiclient.discovery import build
from bs4 import BeautifulSoup
from itertools import count

import configparser
import time, random
import requests
import pandas as pd

In [ ]:
def get_urls(calls, search_query, api_key, cse_id):
    ''' Number of calls is linked to expected number of results. Limit is that a function call can only 
        return a maximum of 100 results and returns 10 results per call. So, if a total of 65 results
        are expected, then calls must be equal to 7. 
    '''
    urls = set()
    start_ = 1
    
    service = build('customsearch', 'v1', developerKey = api_key).cse()
    
    # I switch the default start of itertools.count because I find it easier to start counting from 1.
    for i in count(1):
        if i > calls:
            break
            
        result = service.list(q=search_query, cx=cse_id, start=start_).execute()
        for item in result['items']:
            urls.add(item['link'])
            
        start_ += 10
        time.sleep(random.uniform(1,2))
        
    return urls
        

In [ ]:
def get_article_text(urls):
    article_texts = []
    dates_published = []
    
    for url in urls:
        url_text = requests.get(url).text
        soup = BeautifulSoup(url_text, 'lxml')
        
        article_text = soup.find(article_tag, class_ = article_class).text
        date_published = soup.find(date_tag, class_ = date_class).text
        
        article_texts.append(article_text)
        dates_published.append(date_published)
            
        time.sleep(random.uniform(1,2))
        
        

In [ ]:
def save_results():
    columns = ['Link', 'Link-text', 'Date-published']
    df = pd.DataFrame(links, columns=['Links'])

    return df

In [ ]:
# Read in access details from config file
config = configparser.ConfigParser()
config.read("config.ini")
api_key = config["cse"]["API_key"]
three_news_id = config["cse"]["three_news_id"]
myjoyonline_id = config["cse"]["myjoyonline_id"]
citifmonline_id = config["cse"]["citifmonline_id"]

In [ ]:
links = get_urls(calls=10, search_query="ghana water company limited", api_key=api_key, cse_id=three_news_id)